# Récupération jeu de données

Pour la documentation api Spotify



*   https://spotifypy.readthedocs.io/en/latest/api.html
*   https://github.com/spotipy-dev/spotipy/blob/master/examples/search.py
*   https://pypi.org/project/spotipy/








## Installation de l'api SPOTIFY

In [1]:
 # Installation des packages pour Spotify
#pip install spotipy 

## Import des librairies

In [2]:
from function.utils import tqdm, glob, json

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

## Récupération des tokens

/!\ Attention /!\  
Pour pouvoir récuperer les access tokens de spotify il est impératif d'avoir un compte free ou premium.  
De plus avoir créer son APP.  
Par la suite, vous obtiendrez les client_id et client_id_secret permettant l'accès à l'API et le bon fonctionnement du code.

In [3]:
from function.function import get_token

PATH_TOKEN = "../data/token/*"
PATH_JSON = "../data/json_file/"

liste_txt_file = glob.glob(PATH_TOKEN)

GENIUS_ACCESS_TOKEN = None
SPOTIFY_client_id = None
SPOTIFY_client_secret = None

GENIUS_ACCESS_TOKEN, SPOTIFY_client_id, SPOTIFY_client_secret = get_token(liste_txt_file)


In [4]:
from function.dictionnary import get_dictionnary

dictionnaire_song = get_dictionnary(PATH_JSON)


In [5]:
#identifiant spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIFY_client_id,
                                                           client_secret=SPOTIFY_client_secret))

Liste_artiste_titre = []
nom_artiste = 'drake'
results = sp.search(q=nom_artiste, limit=20) # requete de la recherche
if 'tracks' in results and 'items' in results['tracks']:
    Liste_artiste_titre = [(idx, nom_artiste, data['name']) for idx, data in enumerate(results['tracks']['items'])]

In [ ]:
from function.function import get_lyrics_from_genius
from function.scrapping import scrapping_find_lyrics

headers = {"User-Agent": "Mozilla/5.0"}

for idx, artist_name, song_title in tqdm.tqdm(Liste_artiste_titre):
    lyrics_url = get_lyrics_from_genius(song_title, artist_name, GENIUS_ACCESS_TOKEN)
    if lyrics_url:
        list_lyrics = scrapping_find_lyrics(lyrics_url, headers)
        lyrics_of_song = "\n".join(list_lyrics)
        dictionnaire_song.setdefault(artist_name, {}).setdefault(song_title, {})["lyrics"] = lyrics_of_song
    else:
        print(f"Lyrics de {artist_name} pour '{song_title}' not found on Genius.")
        dictionnaire_song.setdefault(artist_name, {}).setdefault(song_title, {})["lyrics"] = None

 30%|███       | 6/20 [00:21<00:34,  2.49s/it]

Lyrics de drake pour WAIT FOR U (feat. Drake & Tems) not found on Genius.


 45%|████▌     | 9/20 [00:27<00:20,  1.86s/it]

Lyrics de drake pour She Will not found on Genius.


 65%|██████▌   | 13/20 [00:40<00:19,  2.84s/it]

...

 70%|███████   | 14/20 [00:41<00:12,  2.14s/it]

Lyrics de drake pour Fair Trade (with Travis Scott) not found on Genius.


 95%|█████████▌| 19/20 [00:50<00:01,  1.76s/it]

Lyrics de drake pour MIA (feat. Drake) not found on Genius.


100%|██████████| 20/20 [00:53<00:00,  2.65s/it]


In [8]:
with open(PATH_JSON+"dictionnaire_song.json", "w") as json_file:
    json.dump(dictionnaire_song, json_file, indent=4)